# download, prepopulate all the csv into minio

In [13]:
import pandas as pd
# import pyarrow as pa
# import pyarrow.parquet as pq

In [14]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from dask.distributed import Client, LocalCluster
import dask
from dask.distributed import get_task_stream

In [15]:
import time

In [16]:
print('pandas version: %s' % pd.__version__)
print('numpy version: %s' % np.__version__)
print('dask version: %s' % dask.__version__)

pandas version: 1.4.1
numpy version: 1.22.2
dask version: 2022.02.0


https://docs.databricks.com/_static/notebooks/koalas-benchmark-distributed-execution.html?_ga=2.216403934.95291449.1648935555-599276868.1645477063

In [17]:
filename = "taxi_dataset.txt"

In [18]:
with open(filename) as file:
    csv_files = [line.rstrip() for line in file]
# only choose yellow taxis
yellow = list(filter(lambda x: "yellow" in x, csv_files))

In [19]:
# make the list small for now
yellow = ['https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-11.csv']

In [20]:
yellow

['https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-11.csv']

# Configure minio

In [9]:
import getpass

In [10]:
#tag::minio_storage_options[]
# straight from holden's example
minio_storage_options = {
    "key": getpass.getpass(),
    "secret": getpass.getpass(),
    "client_kwargs": {
        "endpoint_url": "http://minio-1602984784.minio.svc.cluster.local:9000",
        "region_name": 'us-east-1'
    },
    "config_kwargs": {"s3": {"signature_version": 's3v4'}},
}
#end::minio_storage_options[]

 ···············································
 ···············································


In [21]:
import re
pattern = '/[^\/]*\.csv$/gm'

for csv_url in yellow:
#     get the filename only so I can sort by month / year
    csv_name = re.findall(r"[^\/]*\.csv$",csv_url)
    df = dd.read_csv(csv_url,dtype={'tolls_amount': 'float64'})

    df = df.repartition(npartitions=4)
    df.to_parquet(f's3://mika-dask-rsch/nyc_{csv_name}',
                  compression="gzip", s
                  torage_options=minio_storage_options, 
                  engine="pyarrow")


RuntimeError: `fastparquet` not installed